In [ ]:
# %%capture
!pip install -q transformers==4.44.2 datasets==2.21.0 evaluate==0.4.2 accelerate==0.34.2 sentencepiece==0.2.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [ ]:
!pip install -U fsspec gcsfs



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.0
    Uninstalling gcsfs-2025.3.0:
      Successfully uninstalled gcsfs-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


In [ ]:
import numpy as np
from typing import Dict, List, Optional

import torch
from datasets import load_dataset, DatasetDict, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)
import evaluate

set_seed(42)
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())


Torch: 2.8.0+cu126 | CUDA: True


In [ ]:
BASE_MODEL = "ai4bharat/IndicBERTv2-MLM-only"  # IndicBERT v2 encoder (hi + bn supported)
MAX_LEN = 256
NUM_LABELS = 3   # entailment, neutral, contradiction
BATCH_SIZE = 8   # per-device
GRAD_ACCUM = 2   # effective batch 16
LR = 2e-5
EPOCHS = 3
FP16 = True


In [ ]:
# Hindi from IndicXNLI (must specify the 'hi' config)
indicxnli_hi = load_dataset("Divyanshu/indicxnli", "hi")

# Build Hindi train/dev
# Most mirrors expose 'train' and 'validation' splits. If not, we create a small dev from train.
if "train" in indicxnli_hi:
    train_hi = indicxnli_hi["train"]
else:
    # Extremely rare fallback: some mirrors only have a single split
    tmp = indicxnli_hi["validation"]
    train_hi = tmp.train_test_split(test_size=0.2, seed=42)["train"]

if "validation" in indicxnli_hi:
    dev_hi = indicxnli_hi["validation"]
else:
    dev_hi = train_hi.train_test_split(test_size=0.05, seed=42)["test"]

# Bangla test from xnli_bn (rename to match field names used for Hindi)
xnli_bn = load_dataset("csebuetnlp/xnli_bn")
test_bn = xnli_bn["test"].rename_columns({"sentence1": "premise", "sentence2": "hypothesis"})

# Quick sanity checks
print("HI splits:", list(indicxnli_hi.keys()))
print("HI example:", {k: train_hi[0][k] for k in ["premise","hypothesis","label"]})
print("BN example:", {k: test_bn[0][k] for k in ["premise","hypothesis","label"]})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/381449 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4895 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2419 [00:00<?, ? examples/s]

HI splits: ['train', 'test', 'validation']
HI example: {'premise': 'अवधारणात्मक रूप से क्रीम स्किमिंग के दो बुनियादी आयाम हैं-उत्पाद और भूगोल।', 'hypothesis': 'उत्पाद और भूगोल क्रीम स्किमिंग का काम करते हैं।', 'label': 1}
BN example: {'premise': 'আসলে, আমি এমনকি এই বিষয়ে চিন্তাও করিনি, কিন্তু আমি এত হতাশ হয়ে পড়েছিলাম যে, শেষ পর্যন্ত আমি আবার তার সঙ্গে কথা বলতে শুরু করেছিলাম', 'hypothesis': 'আমি তার সাথে আবার কথা বলিনি।', 'label': 0}


In [ ]:
label_names = ["entailment", "neutral", "contradiction"]
label_to_id = {n: i for i, n in enumerate(label_names)}

def normalize_label_column(ds):
    # If labels are ClassLabel, map indices via names; else map strings directly.
    if isinstance(ds.features["label"], ClassLabel):
        names = ds.features["label"].names
        remap = {names.index(n): label_to_id[n] for n in names if n in label_to_id}
        def _map(ex):
            ex["label"] = remap.get(ex["label"], ex["label"])
            return ex
        return ds.map(_map)
    else:
        def _map(ex):
            if isinstance(ex["label"], str):
                ex["label"] = label_to_id[ex["label"]]
            return ex
        return ds.map(_map)

train_hi = normalize_label_column(train_hi)
dev_hi   = normalize_label_column(dev_hi)

# xnli_bn labels are ["contradiction","entailment","neutral"]; remap by name
def remap_bn(ex):
    names = test_bn.features["label"].names if isinstance(test_bn.features["label"], ClassLabel) else None
    if names is not None:
        ex["label"] = label_to_id[names[ex["label"]]]
    return ex

test_bn = test_bn.map(remap_bn)

raw = DatasetDict(train=train_hi, validation=dev_hi, test=test_bn)
raw


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/4895 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 392702
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2490
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 4895
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

cfg = AutoConfig.from_pretrained(
    BASE_MODEL,
    num_labels=NUM_LABELS,
    problem_type="single_label_classification",
)

model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, config=cfg)


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-MLM-only and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess(batch):
    return tok(
        batch["premise"],
        batch["hypothesis"],
        truncation=True,
        max_length=MAX_LEN,
    )

cols_to_keep = {"premise", "hypothesis", "label"}
cols_to_remove = [c for c in raw["train"].column_names if c not in cols_to_keep]

tokenized = raw.map(preprocess, batched=True, remove_columns=cols_to_remove)
data_collator = DataCollatorWithPadding(tokenizer=tok)

for split in ["train", "validation", "test"]:
    print(split, tokenized[split][0].keys())


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/4895 [00:00<?, ? examples/s]

train dict_keys(['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])
validation dict_keys(['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])
test dict_keys(['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
!pip install -q scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": float(accuracy_score(labels, preds)),
        "macro_f1": float(f1_score(labels, preds, average="macro")),
    }

In [12]:
from transformers import TrainingArguments, Trainer
import torch

out_dir = "indicbertv2_hi_to_bn_nli"

args = TrainingArguments(
    output_dir=out_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,              # keep disk use low
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    num_train_epochs=EPOCHS,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    fp16=FP16,
    report_to="none",
    dataloader_pin_memory=torch.cuda.is_available(),  # avoids warning on CPU
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tok,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# -----------------------------------------------------------------
# Fix for ValueError: "non-contiguous tensor" when saving
# -----------------------------------------------------------------
def make_model_contiguous(model):
    """Ensures all model weights are contiguous in memory."""
    with torch.no_grad():
        for name, param in model.named_parameters():
            if not param.is_contiguous():
                param.data = param.contiguous()

make_model_contiguous(trainer.model)

# -----------------------------------------------------------------
# Train model
# -----------------------------------------------------------------
train_result = trainer.train()

# -----------------------------------------------------------------
# Optionally save model manually to ensure success
# -----------------------------------------------------------------
trainer.model.save_pretrained("indicbertv2_hi_to_bn_final")
tok.save_pretrained("indicbertv2_hi_to_bn_final")


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.570800,0.576954,0.773494,0.773098
2,0.535700,0.529946,0.792771,0.793614


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.570800,0.576954,0.773494,0.773098
2,0.535700,0.529946,0.792771,0.793614
3,0.360900,0.554495,0.798795,0.799013


('indicbertv2_hi_to_bn_final/tokenizer_config.json',
 'indicbertv2_hi_to_bn_final/special_tokens_map.json',
 'indicbertv2_hi_to_bn_final/tokenizer.json')

In [13]:
results_bn = trainer.evaluate(eval_dataset=tokenized["test"])
print("Zero-shot on Bangla (xnli_bn) =>", results_bn)

Zero-shot on Bangla (xnli_bn) => {'eval_loss': 0.5779005885124207, 'eval_accuracy': 0.7887640449438202, 'eval_macro_f1': 0.7890323197935688, 'eval_runtime': 8.3708, 'eval_samples_per_second': 584.772, 'eval_steps_per_second': 73.111, 'epoch': 3.0}
